0 导入包

In [1]:
# 加载模型
from transformers import pipeline

# 加载数据
from transformers import AutoFeatureExtractor
from datasets import load_dataset
from datasets import Audio


# 随机测试数据
import random

# 播放数据
from IPython.display import Audio as AU # 接收(array，int)，输出音频

# 计算资源
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

In [2]:
# 对比模型地址
ft_model_id = "whisper-fted-common_voice/checkpoint-620"
ori_model_id = "openai/whisper-small"

1 加载数据集

In [3]:
common_voice_test = load_dataset("mozilla-foundation/common_voice_13_0", "zh-CN", split="test") # 超过10个g大小
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16000)) # 调整采样率
common_voice_test = common_voice_test.select_columns([ "audio","sentence" ])
common_voice_test 

d:\env_config\py\lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 10624
})

2 随机挑选数据进行比较

In [4]:
# 随机选择数据
i = random.randint(1,10000)
array = common_voice_test[i]['audio']['array']
# 播放音乐
print(f'{i}答案：',
      common_voice_test[i]['sentence'])
AU(array, rate=16000) # 播放

9998答案： 提出落实全面从严治党要求的工作安排


In [5]:
# ——————————原模型——————————
asr = pipeline("automatic-speech-recognition",model = ori_model_id ,device=device,)
asr(array,generate_kwargs={"task": "transcribe","language": "chinese",},) # 输入音频numpy数组，或文件地址，输出预测结果

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
d:\env_config\py\lib\site-packages\transformers\models\whisper\modeling_whisper.py:691: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'text': '提出落實全面從研製等要求的工作安排'}

In [6]:
# ——————————训练后模型——————————
asr = pipeline("automatic-speech-recognition",model = ft_model_id ,device=device,)
asr(array,generate_kwargs={"task": "transcribe","language": "chinese",},) # 输入音频numpy数组，或文件地址，输出预测结果

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'text': '提出落实全面从严治党要求的工作安排'}

In [7]:
'''
音源错误 1口音发音错误 2短剧错误 3吐字不清

模型结果错误 1不理解知识背景产生的同音不同意字 2 指代不清 3特殊名词写不出正确的字
'''

'\n音源错误 1口音发音错误 2短剧错误 3吐字不清\n\n模型结果错误 1不理解知识背景产生的同音不同意字 2 指代不清 3特殊名词写不出正确的字\n'

3 做成一个demo

In [8]:
import gradio as gr
model_id = "whisper-fted-common_voice/checkpoint-620"
pipe = pipeline("automatic-speech-recognition",model = ft_model_id ,device=device,)

# 定义翻译函数
def transcribe_speech(filepath):
    output = pipe(
        filepath,max_new_tokens=256,
        generate_kwargs={"task": "transcribe","language": "chinese",},  
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]

# 创建Gradio Blocks应用
demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs=gr.Textbox(),
)

file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload", type="filepath"),
    outputs=gr.Textbox(),
)

with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcribe Microphone麦克风", "Transcribe Audio File上传音频"],
    )

demo.launch(debug=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


d:\env_config\py\lib\site-packages\gradio\processing_utils.py:588: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
